In [13]:
# Simplified experimental setup, we will try to experiment with one/more additional crossover operator. We insert our own tau function as well

In [14]:
# Imports
import numpy as np
import inspect
from queue import PriorityQueue
from itertools import combinations
import math

In [26]:
# Vertex data object: identified by name
class Vertex:
    def __init__(self, name):
        self.name = name
        
# Edge data object: edge (frm -> to) with weight function 'tau'
class Edge:
    def __init__(self, frm, to, tau, length, dijkstra_weight=1):
        self.frm = frm
        self.to = to
        self.tau = tau
        self.length = length
        self.dijkstra_weight = dijkstra_weight
        
    # made dnorm absolute value to prevent negative weights. Is this ok?    
    def generate_dijkstra_weight(self, k):
        self.dijkstra_weight = abs(np.random.normal(self.tau(k, self.length), 0.8*self.tau(k, self.length)))
        return self.dijkstra_weight
        
    def __repr__(self):
        tau_str = str(inspect.getsourcelines(self.tau)[0])
        tau_str = tau_str.strip("['\\n']").split(" = ")[1]
        return f'\n({self.frm.name} -> {self.to.name}, {tau_str})'
    
# Graph data object
class Graph:
    def __init__(self):
        self.vertices = []
        self.edges = []
        
    def add_vertex(self, name):
        self.vertices.append(Vertex(name))
        
    def add_vertices(self, num_vertices, start_at_zero=False):
        for i in range(num_vertices):
            self.add_vertex(i+int(not start_at_zero))
            
    def get_vertex(self, name):
        return next(v for v in self.vertices if v.name == name)
    
    def get_edge(self, v1_name, v2_name):
        for e in self.edges:
            if e.frm.name == v1_name and e.to.name == v2_name:
                return e
            elif e.to.name == v1_name and e.frm.name == v2_name:
                return e
#             else:
#                 print('no edge found ', v1_name, ' ', v2_name)
        
        
    def add_edge(self, frm_name, to_name, tau, length):
        assert frm_name in [v.name for v in self.vertices], f'Vertex {frm_name} not in Graph'
        assert to_name in [v.name for v in self.vertices], f'Vertex {to_name} not in Graph'
        frm = self.get_vertex(frm_name)
        to = self.get_vertex(to_name)
        self.edges.append(Edge(frm, to, tau, length))
        
    def get_neighbors(self, vertex_name):
        assert vertex_name in [v.name for v in self.vertices], f'Vertex {vertex_name} not in Graph'
        neighbors = []
        for edge in self.edges:
            if edge.frm.name == vertex_name:
                neighbors.append(edge.to)
            if edge.to.name == vertex_name:
                neighbors.append(edge.frm)
        return neighbors
    
    def generate_dijkstra_weights(self, k):
        for edge in self.edges:
            edge.generate_dijkstra_weight(k)
            
    def get_average_neighbor_count(self):
        avg_count = 0
        for vertex in self.vertices:
            avg_count += len(self.get_neighbors(vertex.name))
        return avg_count / len(self.vertices)
    
    def __str__(self):
        return str(self.edges)

In [27]:
# Test graph parameters
tau_linear = lambda x, l: l*x
tau_constant = lambda x, l: l*2
num_vertices = 7
graph_test = Graph()

# Add vertices
graph_test.add_vertices(8)

# Add edges
graph_test.add_edge(1, 2, tau_linear, 1)
graph_test.add_edge(1, 3, tau_linear, 1)
graph_test.add_edge(1, 4, tau_linear, 1)
graph_test.add_edge(2, 6, tau_linear, 1)
graph_test.add_edge(3, 4, tau_linear, 1)
graph_test.add_edge(3, 8, tau_linear, 1)
graph_test.add_edge(4, 5, tau_linear, 1)
graph_test.add_edge(5, 8, tau_linear, 1)
graph_test.add_edge(6, 7, tau_linear, 1)
graph_test.add_edge(7, 8, tau_linear, 1)



In [28]:
# RandDijkstra
# k is continuous flow of drivers (real number > 0)
def rand_dijkstra(graph, source, target, k, overwrite_weights={}):
#     graph.generate_dijkstra_weights(k)
    
    D = {v.name:(float('inf'), None) for v in (graph.vertices)}
    D[source.name] = (0, None)
    visited = []

    pq = PriorityQueue()
    pq.put((0, source.name))

    while not pq.empty():
        (dist, current_vertex_name) = pq.get()
        visited.append(current_vertex_name)

        for neighbor in graph.get_neighbors(current_vertex_name):
            if neighbor not in visited:
                edge = graph.get_edge(current_vertex_name, neighbor.name)
                weight = None
                
                # Overwrite weights when edge exists in dictionary
                if (edge.frm.name, edge.to.name) in overwrite_weights:
                    weight = overwrite_weights[(edge.frm.name, edge.to.name)]
                elif (edge.to.name, edge.frm.name) in overwrite_weights:
                    weight = overwrite_weights[(edge.to.name, edge.frm.name)]
                else:
                    weight = edge.generate_dijkstra_weight(k)
                old_cost = D[neighbor.name][0]
                new_cost = D[current_vertex_name][0] + weight
                if new_cost < old_cost:
                    pq.put((new_cost, neighbor.name))
                    D[neighbor.name] = (new_cost, current_vertex_name)
                    
    prev = D[target.name][1]
    route_to_target = [target.name]
    while prev != None:
        route_to_target.insert(0, prev)
        prev = D[prev][1]
    
    return D[target.name][0], route_to_target

print(rand_dijkstra(graph_test, graph_test.get_vertex(1), graph_test.get_vertex(8), 100))

(170.20789935037078, [1, 3, 8])


In [ ]:
#mutation operators: tuple of 4 operator functions
def multiple_router_EA(G, s, t, k, n, mu, cStra, mutation_ops, max_it=200):
    P_st = []
    P = []
    diversity_scores, travel_times = [], []
    ex_segment_threshold = int(0.2*max_it)
    ex_segment_incr = None
    weights = [60, 30, 30, 15]
    
    for i in range(mu):
        ind = [rand_dijkstra(G, s, t, k)[1] for _ in range(n)]
        
        #our addition
        for route in ind:
            if not route in P_st:
                P_st.append(route)
        
        P.append(ind)
    for it in range(max_it):
        C = []
        for j in range(int(np.sqrt(mu**2-(mu/2)))):
            # choose uniformly at random chosen
            indices_inds = np.random.choice(range(len(P)), 2, replace=False)
            ind1 = P[indices_inds[0]]
            ind2 = P[indices_inds[1]]
            
            C.append(cStra(ind1, ind2))
        P_ = P.copy()
        for mut_index, ind in enumerate(P_):
            mutations = max(1, np.random.poisson(1.5))
            ops = np.random.choice(mutation_ops, size=mutations, p=weights)
                
            # Because exSegment is too expense: if it is in the list
            # we will discard all other operators
            if ex_segment in ops:
                ops = [ex_segment]
            
            for operator in ops:
                mutated_ind = operator(ind, P_st, G, s, t, k)
                
                # replace 
                P_[mut_index] = mutated_ind
                
        # Update weights
        if it >= 10:
            weights[0] = 30 - it*(29/(max_it-10))
        if travel_times[-1] >= travel_times[max(0, len(travel_times-ex_segment_threshold-1))]:
            if ex_segment_incr is None:
                ex_segment_incr = max_it-it
            weights[3] = 15 + it*(15/ex_segment_incr)
            
        # if no individual in 𝐶 is better than the best in 𝑃 then:
        C_travel_times = []
        for C_ind in C:
            print(P_st, C_ind)
            f = calc_f(C_ind, P_st)
            travel_time = calculate_overall_travel_time(P_st, f, G)
            C_travel_times.append(travel_time)

        P_travel_times = []
        for P_ind in P:
            f = calc_f(P_ind, P_st)
            travel_time = calculate_overall_travel_time(P_st, f, G)
            P_travel_times.append(travel_time)
            
        P__travel_times = []
        for P__ind in P_:
            f = calc_f(P__ind, P_st)
            travel_time = calculate_overall_travel_time(P_st, f, G)
            P__travel_times.append(travel_time)

        if max(C_travel_times) < max(P_travel_times):
            C = []
            C_travel_times = []
        
        #P = best mu individuals in C U P_ U P 
        # We assume that P, P_, C are not sets, so duplicates are NOT removed
        P_union = P + P_ + C 
        P_tt_union = P_travel_times + P__travel_times + C_travel_times
        
        # Sort P_union by P_tt_union
        tuple_P = [(tt, x) for tt, x in sorted(zip(P_tt_union, P_union))][:mu]
        P = [x for tt, x in tuple_P]
        travel_times.append(tuple_P[0][0])
        diversity_scores.append(diversity_score(P[0]))
        
    best_P = P[0]
    
    return best_P, travel_times, diversity_scores
        
        

        
def diversity_score(inds):
    edges_seen = []
    edges_counts = []
    for route in inds:
        for i in range(1,len(route)):
            e = (route[i-1], route[i])
            if not e in edges_seen:
                edges_seen.append(e)
                edges_counts.append(1)
            else:
                index = edges_seen.index(e)
                edges_counts[index] += 1
            
    
    top = [c**2 for c in edges_counts if c>1]
    bottom = [c for c in edges_counts if c==1]
    score = sum(top)/max(1,sum(bottom))
    
    return score
        
        

def exhaustive_crossover(ind1, ind2):
    inds = ind1+ind2
    n = len(ind1)
    
    list_combinations = list(combinations(inds,n))
    list_combinations = [list(x) for x in list_combinations]
    scores = list(map(diversity_score, list_combinations))
    arg = np.argmax(scores)
    
    return list_combinations[arg]

# Mutation Operators

In [30]:
def get_inverse_tf_probs(ind, P_st, G, s, t, k):  
    flows = calc_f(ind, P_st)
    sum_flows = sum(flows)            
    return [1-f/sum_flows for f in flows]

# Helper function to get random index in list for all els
def get_random_indices(list, els):
    indices_list = []
    
    for el in els:
        occurences = [i for i, e in enumerate(list) if e == el]
        indices_list.append(np.random.choice(occurences))
    return indices_list

# Helper function to remove cycles in route
def remove_cycles(route):
    new_route = []
    visited_vertices = []
    for i, v in enumerate(route):
        if v in visited_vertices:
            cycle_start = new_route.index(v)
            new_route = new_route[:cycle_start] + new_route[i:]
        visited_vertices.append(v)
        new_route.append(v)
    return new_route
            
# pain and suffering
def NewRoute(ind, P_st, G, s, t, k):  
    probs = get_inverse_tf_probs(ind, P_st)
    
    if 1 in probs:
        for index, value in enumerate(probs):
            if value < 1:
                P_st_index = index 
    else:
        P_st_index = np.random.choice(range(len(P_st)), p=probs)
    
    route = P_st[P_st_index]
    
    print(route)
    route_index = ind.index(route)
    
    ind[route_index] = rand_dijkstra(G, s, t, k)[1]
    
    return ind


# Select a random route in ind. Select the highest flow edge in the route and find a route which
# circumvents the 'crowded' edge
def Roundabout(ind, P_st, G, s, t, k):
    f = calc_f(ind,P_st)
    index = np.random.choice(range(len(ind)))
    route = ind[index]
    
    max_edge = 0
    max_edge_flow = 0
    for vertex_i in range(len(route)-1):
        frm = route[vertex_i]
        to = route[vertex_i+1]
        edge = G.get_edge(frm,to)
        
        
        edge_flow = get_edge_traffic_flow(P_st,f,edge)
        if edge_flow >= max_edge_flow:
            max_edge = edge
            max_edge_flow = edge_flow
            
    frm = getattr(getattr(max_edge,'frm'),'name')
    
    neighbor_vertices = G.get_neighbors(frm)
    while leq_two_neighbors(neighbor_vertices):
        frm = get_previous_vertex(route,frm)
        neighbor_vertices = G.get_neighbors(frm)
    
    neighbors_names = [getattr(vert,'name') for vert in neighbor_vertices]
    #remove vertices which are connected through the crowded edge and the previous edge
    prev = get_previous_vertex(route,frm)
    to = getattr(getattr(max_edge,'to'),'name')
    
    neighbors_names.remove(prev)
    neighbors_names.remove(to)
    
    new_vertex_name = np.random.choice(neighbors_names)
    print('frm, new_vertex_name',frm, new_vertex_name, neighbors_names)
    new_vertex = G.get_vertex(new_vertex_name)
    frm_index = route.index(frm)
    route = route[:frm_index]
    redirection = rand_dijkstra(G, new_vertex, t, k)[1]
    
    new_route = route+[frm]+redirection
    print(route, redirection)
    if not new_route in P_st:
        P_st.append(new_route)
    
    ind[index] = new_route
    
    return ind
    
# Mutation operator RandomP implementation
def random_P(ind, P_st, G, s, t, k):
    new_ind = ind.copy()
    
    probs = get_inverse_tf_probs(new_ind, P_st)
    route_subset_size = np.random.randint(1, len(new_ind))
    P_st_indices = np.random.choice(range(len(P_st)), size=route_subset_size, p=probs)
    route_subset = [P_st[P_st_index] for P_st_index in P_st_indices]
    route_indices = get_random_indices(ind, route_subset)

    for i, route in enumerate(route_subset):
        r = len(route)
        start_node_index = np.random.randint(r-2)
        subroute_length = max(min(abs(int(np.random.normal(0.25*r, 0.5*r))), r-start_node_index-1), 2)
        end_node_index = start_node_index + subroute_length
        subroute = route[start_node_index:end_node_index]
        overwrite_weights = {(edge.frm.name, edge.to.name):1000 for edge in get_edges(G, subroute)}
        new_subroute = rand_dijkstra(G, G.get_vertex(subroute[0]), G.get_vertex(subroute[-1]), k, overwrite_weights)[1]
        new_route = route[:start_node_index] + new_subroute + route[end_node_index:]
        new_route = remove_cycles(new_route)
        new_ind[route_indices[i]] = new_route
        
    return new_ind

# Mutation operator ExSegment implementation
def ex_segment(ind, P_st, G, s, t, k):
    new_ind = ind.copy()
    
    one_dim_list = [tuple(r) for r in ind]    
    num_diff_routes = len(set(one_dim_list))
    num_pair_els = np.random.randint(1, (num_diff_routes//2)+1) * 2
    route_pair_elements = np.random.choice(range(len(one_dim_list)), size=num_pair_els, replace=False)
    route_pairs = [(ind[route_pair_elements[i]], ind[route_pair_elements[i+1]]) for i in range(0, num_pair_els, 2)]
    
    # route_pairs = [([1, 2, 3, 4, 5, 6], [1, 7, 8, 9, 10, 3, 4, 11, 12, 13, 6])]
    
    for i, (r1, r2) in enumerate(route_pairs):
        if r1 != r2:
            r1_ind = route_pair_elements[i]
            r2_ind = route_pair_elements[i+1]
            r1_len = len(r1)
            r2_len = len(r2)
            # Compute divergence + goto index pairs
            div_pairs, goto_pairs = [], []
            for i, v1 in enumerate(r1):
                for j, v2 in enumerate(r2):
                    if v1 == v2 and i+1 < r1_len and j+1 < r2_len and r1[i+1] != r2[j+1]:
                        div_pairs.append((i, j))
                        break
                    elif v1 == v2 and i-1 > 0 and j-1 > 0 and r1[i-1] != r2[j-1]:
                        goto_pairs.append((i, j))
                        break
            # Randomly select div pair
            div_pair = div_pairs[np.random.choice(range(len(div_pairs)))]
            # Select goto pair occuring after div pair
            goto_pairs_after = [(i1, i2) for i1, i2 in goto_pairs if i1 > div_pair[0] and i2 > div_pair[1]]
            goto_pair = goto_pairs_after[np.random.choice(range(len(goto_pairs_after)))]
            # Swap
            new_r1 = r1[:div_pair[0]] + r2[div_pair[1]:goto_pair[1]] + r1[goto_pair[0]:]
            new_r2 = r2[:div_pair[1]] + r1[div_pair[0]:goto_pair[0]] + r2[goto_pair[1]:]
            new_ind[r1_ind] = new_r1
            new_ind[r2_ind] = new_r2
    
    return new_ind

def leq_two_neighbors(lst):
    return len(lst) <= 2

def get_previous_vertex(route, vertex):
    index = route.index(vertex)
    if index == 0:
        print('get_previous_vertex: is already the first vertex')
        return route[index]
    
    return route[index-1]
    

In [31]:
# Traffic Flow (f) = list of integers mapping index of route to drivers per unit

# Helper function that retrieves all edges in a route
def get_edges(graph, route):
    edges = []
    for i in range(len(route)-1):
        edges.append(graph.get_edge(route[i], route[i+1]))
    return edges


# Helper function to get traffic flow of edge given routes (P_st), traffic flow (f) and edge
def get_edge_traffic_flow(P_st, f, edge):
    value = 0
    for i, p in enumerate(P_st):
        for j in range(len(p)-1):
            if (p[j] == edge.frm.name and p[j+1] == edge.to.name) or (p[j+1] == edge.frm.name and p[j] == edge.to.name):
                value += f[i]
                break
    return value

# Function to calculate travel time
def calculate_travel_time(P_st, f, p, i, graph):
    f_p = f[i]
    tau_p = sum([edge.tau(get_edge_traffic_flow(P_st, f, edge), edge.length) for edge in get_edges(graph, p)])
    return f_p * tau_p

# Function to calculate overall travel time (C)
def calculate_overall_travel_time(P_st, f, graph):
    overall_travel_time = 0
    for i, p in enumerate(P_st):
        overall_travel_time += calculate_travel_time(P_st, f, p, i, graph)
    return overall_travel_time


# Possibly computationally inefficient if P_st is very large compared to ind
def calc_f(ind, P_st):
    f = []
    for existing_route in P_st:
        occurrence = 0
        for route in ind:
            if existing_route == route:
                occurrence += 1
        f.append(occurrence)
    return f

In [32]:
# Test graph parameters
tau_linear = lambda x, l: l*x
tau_constant = lambda x, l: l*1
num_vertices = 5
G = Graph()

# Add vertices
G.add_vertices(num_vertices)


# Add edges
G.add_edge(1, 2, tau_linear, 1)
G.add_edge(1, 3, tau_linear, 1)
G.add_edge(2, 4, tau_linear, 1)
G.add_edge(3, 4, tau_linear, 1)
G.add_edge(4, 5, tau_linear, 1)

s = G.get_vertex(1)
t = G.get_vertex(5)

k = 3

mu = 2

cStra = exhaustive_crossover

mutation_ops = [Roundabout]


P_st = [[1,2,4,5],[1,3,4,5]]
ind = [[1,2,4,5],[1,2,4,5],[1,3,4,5]]

ind = ex_segment(ind, P_st, G, s, t, k)

# f = calc_f(ind, P_st)
# calculate_overall_travel_time(P_st, f, G)
# lst = exhaustive_crossover(ind, ind)
# print(lst)

In [9]:
multiple_router_EA(G, s, t, k, k, mu, cStra, mutation_ops, max_it=200)

frm, new_vertex_name 4 3 [3]
[1, 2] [3, 4, 5]
frm, new_vertex_name 4 2 [2]
[1, 3] [2, 1, 3, 4, 5]
[[1, 3, 4, 5], [1, 2, 4, 5], [1, 2, 4, 3, 4, 5], [1, 3, 4, 2, 1, 3, 4, 5]] [[1, 3, 4, 5], [1, 3, 4, 5], [1, 3, 4, 5]]
frm, new_vertex_name 4 3 [3]
[1, 2] [3, 4, 5]
frm, new_vertex_name 4 3 [3]
[1, 2] [3, 4, 5]
frm, new_vertex_name 4 3 [3]
[1, 2] [3, 4, 5]
frm, new_vertex_name 4 3 [3]
[1, 2] [3, 1, 2, 4, 5]
frm, new_vertex_name 4 3 [3]
[1, 2] [3, 4, 5]
[[1, 3, 4, 5], [1, 2, 4, 5], [1, 2, 4, 3, 4, 5], [1, 3, 4, 2, 1, 3, 4, 5], [1, 2, 4, 3, 1, 2, 4, 5]] [[1, 2, 4, 3, 4, 5], [1, 2, 4, 5], [1, 2, 4, 3, 4, 5]]
frm, new_vertex_name 4 3 [3]
[1, 2] [3, 4, 5]
frm, new_vertex_name 4 3 [3]
[1, 2] [3, 4, 5]
frm, new_vertex_name 4 2 [2]
[1, 3] [2, 4, 5]
[[1, 3, 4, 5], [1, 2, 4, 5], [1, 2, 4, 3, 4, 5], [1, 3, 4, 2, 1, 3, 4, 5], [1, 2, 4, 3, 1, 2, 4, 5], [1, 3, 4, 2, 4, 5]] [[1, 2, 4, 3, 4, 5], [1, 2, 4, 3, 1, 2, 4, 5], [1, 2, 4, 3, 1, 2, 4, 5]]
frm, new_vertex_name 4 2 [2]
[1, 3] [2, 4, 5]
frm, new_verte

[[1, 3, 4, 2, 4, 5], [1, 2, 4, 3, 4, 5], [1, 2, 4, 3, 4, 5]]

In [33]:
# Tau functions
road_30 = lambda x, l: (50/l)*x**2
road_50 = lambda x, l: (25/l)*x**2
road_80 = lambda x, l: (10/l)*x**3
road_100 = lambda x, l: (2/l)*x**5

In [124]:
# Road network graph generator
def generate_road_network(
        num_vertices=200, 
        desired_neighbor_count=3.5, 
        primary_arch_alpha=16, 
        secondary_arch_alpha=32, 
        sub_arch_min=2, 
        sub_arch_max=5, 
        main_road=road_100,
        primary_arch_road=road_80,
        secondary_arch_road=road_50,
        random_road=road_30,
        seed=42
    ):
    np.random.seed(seed)
    G = Graph()
    G.add_vertices(num_vertices, start_at_zero=True)
    cur_vertex = 0
    
    # Create main road (source -> target)
    main_road_length = num_vertices//8
    for i in range(main_road_length):
        road_length = np.random.randint(5, 10)
        G.add_edge(i, i+1, main_road, road_length)
    cur_vertex = main_road_length + 1
    
    # Create primary + secondary arches
    primary_arch_length = num_vertices // primary_arch_alpha
    secondary_arch_length = num_vertices // secondary_arch_alpha
    max_arch_size = primary_arch_length + sub_arch_max * secondary_arch_length
    while cur_vertex + max_arch_size < num_vertices:
        # Add primary arch
        start_node, end_node = np.random.choice(range(main_road_length), size=2, replace=False)
        G.add_edge(start_node, cur_vertex, primary_arch_road, np.random.randint(3, 5))
        G.add_edge(cur_vertex + primary_arch_length, end_node, primary_arch_road, np.random.randint(3, 5))
        for j in range(cur_vertex, cur_vertex + primary_arch_length):
            road_length = np.random.randint(3, 5)
            G.add_edge(j, j+1, primary_arch_road, road_length)
        # Add secondary arches
        num_secondary_arches = np.random.randint(sub_arch_min, sub_arch_max)
        primary_arch_start = cur_vertex
        primary_arch_end = primary_arch_start + primary_arch_length
        cur_vertex = primary_arch_end + 1
        for k in range(num_secondary_arches):
            arch_start_node, arch_end_node = np.random.choice(range(primary_arch_start, primary_arch_end), size=2, replace=False)
            G.add_edge(arch_start_node, cur_vertex, secondary_arch_road, np.random.randint(3, 5))
            G.add_edge(cur_vertex + secondary_arch_length, arch_end_node, secondary_arch_road, np.random.randint(3, 5))
            for l in range(cur_vertex, cur_vertex + secondary_arch_length):
                road_length = np.random.randint(3, 5)
                G.add_edge(l, l+1, secondary_arch_road, road_length)
            cur_vertex = cur_vertex + secondary_arch_length + 1
            
            
    # Add connections
    for vertex_index in range(cur_vertex, num_vertices):
        tiny_arch_start_node, tiny_arch_end_node = np.random.choice(range(0, cur_vertex), size=2, replace=False)
        G.add_edge(tiny_arch_start_node, vertex_index, random_road, np.random.randint(3, 5))
        G.add_edge(vertex_index, tiny_arch_end_node, random_road, np.random.randint(3, 5))
    
    # Add random connections
    while G.get_average_neighbor_count() <= desired_neighbor_count:
        v1_node, v2_node = np.random.choice(range(num_vertices), size=2, replace=False)
        G.add_edge(v1_node, v2_node, random_road, np.random.randint(3, 5))
        
    return G

In [125]:
G = generate_road_network()